<a href="https://colab.research.google.com/github/LeeSeungHyun4839/ML-Simulator/blob/master/ML_simulator_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import datasets
from torchvision import transforms
from torch import nn
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.autograd import Variable

BATCH_SIZE = 128
LEARNING_RATE = 0.002
EPOCHS = 1000
SNR = 0
I = 3
L = 3

In [ ]:
import csv
import numpy as np
channel_true_Re = []
with open('./SCM_channel_true_Re.csv') as csvfile:
  Channel_true_Re_temp = csv.reader(csvfile, delimiter = ",")
  for row in Channel_true_Re_temp:
    channel_true_Re.append((row))
channel_true_Re = np.array(channel_true_Re, float)

channel_true_Im = []
with open('./SCM_channel_true_Im.csv') as csvfile:
  Channel_true_Im_temp = csv.reader(csvfile, delimiter = ",")
  for row in Channel_true_Im_temp:
    channel_true_Im.append((row))
channel_true_Im = np.array(channel_true_Im, float)

print(np.shape(channel_true_Im))

(2500, 64)


In [ ]:
filename_Re = './SCM_channel_SNR' + str(SNR) + '_Re.csv'
filename_Im = './SCM_channel_SNR' + str(SNR) + '_Im.csv'

channel_SNR_Re = []
with open(filename_Re) as csvfile:
  Channel_SNR_Re_temp = csv.reader(csvfile, delimiter = ",")
  for row in Channel_SNR_Re_temp:
    channel_SNR_Re.append((row))
channel_SNR_Re = np.array(channel_SNR_Re, float)

channel_SNR_Im = []
with open(filename_Im) as csvfile:
  Channel_SNR_Im_temp = csv.reader(csvfile, delimiter = ",")
  for row in Channel_SNR_Im_temp:
    channel_SNR_Im.append((row))
channel_SNR_Im = np.array(channel_SNR_Im, float)

In [ ]:
# Training data set
train = []

Train_Re = channel_SNR_Re[:2048, :]
Train_Im = channel_SNR_Im[:2048, :]

for i in range(2048-I):
  train_re = Train_Re[i:I+i,:].flatten()
  train_im = Train_Im[i:I+i,:].flatten()

  train_i = np.concatenate((train_re, train_im))
  train.append(train_i)

train = np.array(train)

label = np.concatenate((channel_SNR_Re[I:2048, :], \
                             channel_SNR_Im[I:2048, :]), axis = 1)

print(np.shape(train))
print(np.shape(label))

(2045, 384)
(2045, 128)


In [ ]:
# Test data set
Test_Re = channel_SNR_Re[2200-I:2299, :]
Test_Im = channel_SNR_Im[2200-I:2299, :]

test = []

for i in range(100):
  test_re = Test_Re[i:I+i,:].flatten()
  test_im = Test_Im[i:I+i,:].flatten()

  test_i = np.concatenate((test_re, test_im))
  test.append(test_i)

test = np.array(test)

label_test = np.concatenate((channel_true_Re[2200:2300, :], \
                             channel_true_Im[2200:2300, :]), axis = 1)

print(np.shape(test))
print(np.shape(label_test))

(100, 384)
(100, 128)


In [ ]:
test = torch.from_numpy(test).float()
label_test = torch.from_numpy(label_test).float()

In [ ]:
train = torch.from_numpy(train).float()
label = torch.from_numpy(label).float()

print(np.shape(train))
print(np.shape(label))

dataset_train = TensorDataset(train, label)
data_loader = DataLoader(dataset_train, batch_size = BATCH_SIZE, shuffle = True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'


torch.Size([2045, 384])
torch.Size([2045, 128])


In [ ]:
class MLP(nn.Module):
  def __init__(self):
    super(MLP, self).__init__()
    self.fcInput = nn.Linear(I*2 * 64, 512, bias = True)
    self.fcHidden = nn.Linear(512, 512, bias = True)
    self.fcOutput = nn.Linear(512, 128, bias = True)
    
  def forward(self, input, L):
    output = self.fcInput(input)
    if L > 1:
        for i in range(L-1):
            output = self.fcHidden(output)
    output = self.fcOutput(output)

    return output

In [ ]:
model = MLP().to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
loss_ft = nn.MSELoss().to(device)

for epoch in range(EPOCHS):
  losses = []

  for X_batch, Y_batch in data_loader:
    X_batch = X_batch.to(device)
    Y_batch = Y_batch.to(device)
    optimizer.zero_grad()
    pred = model(X_batch, L)
    loss = loss_ft(pred, Y_batch)
    loss.backward()
    optimizer.step()

    losses.append(loss.cpu().detach().numpy())

  print(f'Epoch {epoch+1:3d} :: Loss is {np.array(losses).mean():.6f}')


Epoch   1 :: Loss is 0.011724
Epoch   2 :: Loss is 0.009445
Epoch   3 :: Loss is 0.008929
Epoch   4 :: Loss is 0.008694
Epoch   5 :: Loss is 0.008549
Epoch   6 :: Loss is 0.008441
Epoch   7 :: Loss is 0.008385
Epoch   8 :: Loss is 0.008344
Epoch   9 :: Loss is 0.008316
Epoch  10 :: Loss is 0.008240
Epoch  11 :: Loss is 0.008223
Epoch  12 :: Loss is 0.008197
Epoch  13 :: Loss is 0.008200
Epoch  14 :: Loss is 0.008173
Epoch  15 :: Loss is 0.008178
Epoch  16 :: Loss is 0.008165
Epoch  17 :: Loss is 0.008147
Epoch  18 :: Loss is 0.008157
Epoch  19 :: Loss is 0.008140
Epoch  20 :: Loss is 0.008126
Epoch  21 :: Loss is 0.008110
Epoch  22 :: Loss is 0.008094
Epoch  23 :: Loss is 0.008075
Epoch  24 :: Loss is 0.008079
Epoch  25 :: Loss is 0.008064
Epoch  26 :: Loss is 0.008062
Epoch  27 :: Loss is 0.008045
Epoch  28 :: Loss is 0.008064
Epoch  29 :: Loss is 0.008048
Epoch  30 :: Loss is 0.008044
Epoch  31 :: Loss is 0.008047
Epoch  32 :: Loss is 0.008035
Epoch  33 :: Loss is 0.008031
Epoch  34 

In [ ]:
with torch.no_grad():
    test = test.to(device)
    label_test = label_test.to(device)
    prediction = model(test, L)
    h = label_test
    h_hat = prediction
    error = 0
    for i in range(100):
        h_i = h[i,:]
        h_hat_i = h_hat[i,:]
        error += torch.sum((h_i-h_hat_i)**2) / torch.sum((h_i)**2) /100

    error = error.float()
    print('NMSE =', error)

    dBerror = 10 * torch.log10(error)

    print('Decibel NMSE =', dBerror)

NMSE = tensor(0.4155, device='cuda:0')
Decibel NMSE = tensor(-3.8146, device='cuda:0')
